In [ ]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.enable_eager_execution()


import random
import gym
import numpy as np
from collections import deque
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pylab
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import os

ENV_NAME = "CartPole-v1"

GAMMA = 0.1
LEARNING_RATE = 0.001

MEMORY_SIZE = 100000
MIN_MEMORY_SIZE = 200
BATCH_SIZE = 64

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.9995

from environment.environment import GridWorld

now = '30x40_8POI'

filling = True
scores = []
averages = []
episodes = []

debug_mode = False
show_graph_every = 1
means = False
resume = False


env = GridWorld(show_graph_every, debug_mode, means)


size = 1200



class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX
        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)
        if resume:
            self.model = keras.models.load_model('models/'+now+'.h5', custom_objects=None, compile=True)
            print('Model loaded')
            self.exploration_rate = EXPLORATION_MAX
            self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))
            return
        self.model = Sequential()
        self.model.add(Dense(size, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(size, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))
        print('Model created')

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):        
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def save(self):
        self.model.save('models/'+now+'.h5')   
        print('model saved')
    
    def experience_replay(self, run):
        if len(self.memory) < MIN_MEMORY_SIZE:
            return
        global filling
        filling = False
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)                
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)


def cartpole():
    env = GridWorld(show_graph_every, debug_mode, means)
    #observation_space = env.observation_space.shape[0]
    action_space = 6
    observation_space = size
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            action = dqn_solver.act(state)
            observe_past, reward, total_reward, state_next, terminal = env.step(action)
            #reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal or (not filling and step == 20):
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate)[:4] + ", score: " + str(total_reward)[:4] + ' step: '+ str(step))
                PlotModel(total_reward, run)                
                break
            dqn_solver.experience_replay(run)
        if run % 20 == 0:
            dqn_solver.save()



def cartpole_test():
    #observation_space = env.observation_space.shape[0]
    global show_graph_every
    show_graph_every = 1
    env = GridWorld(show_graph_every, debug_mode, means)
    action_space = 5
    observation_space = size
    model = keras.models.load_model('models/'+now+'.h5', custom_objects=None, compile=True)
    print('Model loaded')
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            action = np.argmax(model.predict(state)[0])
            observe_past, reward, total_reward, state_next, terminal = env.step(action)
            state_next = np.reshape(state_next, [1, observation_space])
            state = state_next
            if terminal:
                print ("Test: " + str(run) + ", score: " + str(total_reward)[:4] + ' step: '+ str(step))
                break            
            
            
            
            
            
def PlotModel(score, episode_number):
    scores.append(score)
    averages.append(sum(scores[-50:]) / len(scores[-50:]))
    episodes.append(episode_number)
    pylab.plot(episodes, scores, 'b')
    pylab.plot(episodes, averages, 'r')
    pylab.ylabel('Score', fontsize=18)
    pylab.xlabel('Games', fontsize=18)
    file_name = "breakout_model_{}".format(now)
    name = file_name + '.png'
    try:
        if not os.path.exists('training_images'): os.makedirs('training_images')
        pylab.savefig('training_images/'+name)

    except OSError as e:
        print(e)
        pass
    return    
            
            
            
            
            
            
            
            
            
            
            
            
            

if __name__ == "__main__":
    cartpole()
    #cartpole_test()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Model created
Run: 1, exploration: 1.0, score: -64. step: 100
Run: 2, exploration: 1.0, score: -68. step: 100
Run: 3, exploration: 0.99, score: -13. step: 20
Run: 4, exploration: 0.98, score: -7.8 step: 20
Run: 5, exploration: 0.97, score: -14. step: 20
Run: 6, exploration: 0.96, score: -6.2 step: 20
Run: 7, exploration: 0.95, score: -15. step: 20
Run: 8, exploration: 0.94, score: -11. step: 20
Run: 9, exploration: 0.93, score: -6.1 step: 20
Run: 10, exploration: 0.92, score: -16. step: 20
Run: 11, exploration: 0.91, score: -4.0 step: 20
Run: 12, exploration: 0.90, score: -10. step: 20
Run: 13, exploration: 0.90, score: -7.5 step: 20
Run: 14, exploration: 0.89, score: -10. step: 20
Run: 15, exploration: 0.88, score: 28.7 step: 12
Run: 16, exploration: 0.87, score: -11. step: 20
Run: 17, exploration: 0.87, score: 25.4 step: 19
Run: 18, exploration: 0.86, score: -9.9 step: 20
Run: 19, exploration: 0.85, 

In [ ]:
import random
random.randint(0,1)